# DS-SF-34 | Unit Project | 3 | Machine Learning Modeling and Executive Summary | Starter Code

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.  You will summarize and present your findings and the methods you used.

In [52]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf

from sklearn import linear_model

In [89]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'dataset-ucla-admissions.csv'))
df.dropna(inplace = True)
  # 397 records
df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [57]:
# TODO
pd.crosstab(df.prestige, df.admit, dropna = False)  

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part B.  Feature Engineering

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [87]:
# TODO
prestige_df = pd.get_dummies(df.prestige, prefix='prestige')
prestige_df.rename(columns = {'prestige_1.0': 'prestige_1',
    'prestige_2.0': 'prestige_2',
    'prestige_3.0': 'prestige_3',
    'prestige_4.0': 'prestige_4'}, inplace = True)
prestige_df
df = df.join([prestige_df])   
df
df.drop(['prestige'], axis=1, inplace=True)

> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: TODO
We only need 3 of the 4 binary variables for modeling, because the value of the 4th variable can be derived from the values of the 3 remaining variables.

> ### Question 4.  Why are we doing this?

Answer: TODO
We need to convert "prestige" to binary variables, because it is not true that the difference between prestige=1 and prestige=2 is the same as the difference between prestige=2 and prestige=3, etc.  The values of "prestige" are not truly linear.

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [59]:
# TODO
# added prestige fields above
df.drop(['prestige_1'], axis = 1, inplace = True)
df

,admit,gre,gpa,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1
...,...,...,...,...,...,...
395,0,620.0,4.00,1,0,0
396,0,560.0,3.04,0,1,0
397,0,460.0,2.63,1,0,0
398,0,700.0,3.65,1,0,0


## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [13]:
# TODO
pd.crosstab(df.prestige[df.prestige==1], df.admit, dropna = False)  

admit,0,1
prestige,,
1.0,28,33


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [ ]:
# TODO
Odds = prob(admit with most prestige) / 1 - prob(admit with most prestige) = 33/28 = 1.18

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [14]:
# TODO
pd.crosstab(df.prestige[df.prestige<>1], df.admit, dropna = False)  
# odds = (53+28+12) / (95+93+55) = 93 / 243 = 0.38

admit,0,1
prestige,,
2.0,95,53
3.0,93,28
4.0,55,12


> ### Question 9.  Finally, what's the odds ratio?

In [ ]:
# TODO
Odds ratio = odds(getting into grad school from #1 school) / odds(getting into grad school NOT from #1 school)
                  = 1.18 / 0.38 = 3.1

> ### Question 10.  Write this finding in a sentence.

Answer: TODO

If you go to a top ranked college, you are 3.1 times more likely to get into UCLA than if you attended a non-top ranked college.

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

In [ ]:
# TODO
Odds of being admitted to UCLA for students who attended the least prestigious colleges:  12/55 = .22
Odds of being admitted to UCLA for students who did NOT attend least prestigious college: (33+53+28) / (28+95+93) = .53
Odds ratio  = .22 / .53 = .42.   
If you go to the lowest ranked college, you are less than half as likely to get into UCLA
than if you had attended a higher ranking college

Answer: TODO

## Part D. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [74]:
names_X = ['gre', 'gpa', 'prestige_2', 'prestige_3','prestige_4']

def X_c(df):
    X = df[ names_X ]
    c = df.admit
    return X, c

X, c = X_c(df)

In [60]:
X['intercept']=1.0
X

/Users/seanmcghee/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,gre,gpa,prestige_2,prestige_3,prestige_4,intercept
0,380.0,3.61,0,1,0,1.0
1,660.0,3.67,0,1,0,1.0
2,800.0,4.00,0,0,0,1.0
3,640.0,3.19,0,0,1,1.0
4,520.0,2.93,0,0,1,1.0
...,...,...,...,...,...,...
395,620.0,4.00,1,0,0,1.0
396,560.0,3.04,0,1,0,1.0
397,460.0,2.63,1,0,0,1.0
398,700.0,3.65,1,0,0,1.0


In [61]:
logit = smf.Logit(c,X)

  # fit the model
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


> ### Question 13.  Print the model's summary results.

In [62]:
# TODO
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 07 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        13:18:04   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2    -0.6801      0.317     -2.146      0.0

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

The odds ratios for the different features are the e^(coef) for each feature.  Their confidence intervals are displayed at the far right under the columns 0.025 and 0.975.

> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: TODO
e^(-0.6801) = .5.  The odds of being admitted to UCLA is half as likely if one attending a second rank college vs a top ranked college.


> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer: TODO
e^(0.7793) = 2.2.  The odds of being admitted to UCLA is 2.2 more likely if your GPA is 1 grade point higher.

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [73]:
# TODO
  # Tier 1  75% chance of getting into UCLA
#names_X = ['gre', 'gpa', 'prestige_2', 'prestige_3','prestige_4']
X_test=[[800,4,0,0,0,1]]   
result.predict(X_test)

array([ 0.73403998])

In [68]:
  # Tier 2  58% chance of getting into UCLA
X=[[800,4,1,0,0,1]]   
result.predict(X_test)

array([ 0.58299512])

In [69]:
  # Tier 3  42% chance of getting into UCLA
X=[[800,4,0,1,0,1]]   
result.predict(X_test)

array([ 0.41983282])

In [71]:
  # Tier 4  37% chance of getting into UCLA
X=[[800,4,0,0,1,1]]   
result.predict(X_test)

array([ 0.36860803])

Answer: TODO

## Part E. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [78]:
clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [75]:
names_X = ['gre', 'gpa', 'prestige_2', 'prestige_3','prestige_4']

def X_c(df):
    X = df[ names_X ]
    c = df.admit
    return X, c

X, c = X_c(df)
print X

       gre   gpa  prestige_2  prestige_3  prestige_4
0    380.0  3.61           0           1           0
1    660.0  3.67           0           1           0
2    800.0  4.00           0           0           0
3    640.0  3.19           0           0           1
4    520.0  2.93           0           0           1
..     ...   ...         ...         ...         ...
395  620.0  4.00           1           0           0
396  560.0  3.04           0           1           0
397  460.0  2.63           1           0           0
398  700.0  3.65           1           0           0
399  600.0  3.89           0           1           0

[397 rows x 5 columns]


In [79]:
# TODO
model = linear_model.LogisticRegression(C=10**2).\
    fit(X,c)

print model.intercept_
print model.coef_

[-3.51478687]
[[ 0.00215822  0.67315496 -0.62882239 -1.25222745 -1.56879212]]


> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [80]:
# TODO
print zip(names_X, np.exp(model.coef_[0]) - 1)

# the odds ratios from the statsmodel were
# ==============================================================================
#                  coef    std err          z      P>|z|      [0.025      0.975]
# ------------------------------------------------------------------------------
# gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
# gpa            0.7793      0.333      2.344      0.019       0.128       1.431
# prestige_2    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
# prestige_3    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
# prestige_4    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
# intercept     -3.8769      1.142     -3.393      0.001      -6.116      -1.638
# ==============================================================================

# the values from sklearn are within the 95% confidence interval of the values 
# from statsmodel, but there are definitely not the same

[('gre', 0.0021605460441196378), ('gpa', 0.96041258790256867), ('prestige_2', -0.46678064340542613), ('prestige_3', -0.7141326687531091), ('prestige_4', -0.79170337207758157)]


Answer: TODO

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [81]:
# TODO
  # Tier 1  71% chance of getting into UCLA
predict_X = [[800, 4, 0, 0, 0]]
print model.predict(predict_X)
print model.predict_proba(predict_X)

[1]
[[ 0.28814605  0.71185395]]


In [82]:
  # Tier 1  57% chance of getting into UCLA
predict_X = [[800, 4, 1, 0, 0]]
print model.predict(predict_X)
print model.predict_proba(predict_X)

[1]
[[ 0.43153702  0.56846298]]


In [83]:
  # Tier 1  41% chance of getting into UCLA
predict_X = [[800, 4, 0, 1, 0]]
print model.predict(predict_X)
print model.predict_proba(predict_X)

[0]
[[ 0.58608936  0.41391064]]


In [84]:
  # Tier 1  34% chance of getting into UCLA
predict_X = [[800, 4, 0, 0, 1]]
print model.predict(predict_X)
print model.predict_proba(predict_X)

[0]
[[ 0.66024514  0.33975486]]


Answer: TODO

## Part F.  Executive Summary

> ## Question 21.  Introduction
>
> Write a problem statement for this project.

Answer: TODO

The goal of this project is to determine which factors influence whether or not a college student will be accepted to UCLA for graduate school.  We will analyze the impact that GRE scores, GPA, and prestige of the undergraduate college/university attended have on UCLA graduate school admissions.  

> ## Question 22.  Dataset
>
> Write up a description of your data and any cleaning that was completed.

Answer: TODO
We have UCLA admissions data for 400 students; unfortunately we do not have the time frame for these data.  

The data fields were:  GRE scores, GPA, and prestige.  GRE score is a value between 0 and 800; here GRE scores were between 220 and 800.  GPA is a value between 0 and 4; here the GPA data ranged from 2.26 to 4.0.  Prestige is an integer between 1 and 4, where 1 is the highest rank undergraduate institution, and 4 is the lowest rank.  The most common value for prestige was 2.

We removed 3 records with null values, resulting in a final dataset of 397 records.

> ## Question 23.  Demo
>
> Provide a table that explains the data by admission status.

Answer: TODO

In [90]:
pd.crosstab(df.prestige, df.admit, dropna = False)  

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


In [91]:
pd.crosstab(df.gre, df.admit, dropna = False)  

admit,0,1
gre,,
220.0,1,0
300.0,2,1
340.0,3,1
360.0,4,0
380.0,8,0
...,...,...
720.0,8,3
740.0,7,4
760.0,1,4


> ## Question 24.  Methods
>
> Write up the methods used in your analysis.

Answer: TODO
We used logistic regression to predict the probability of the a student being admitted to UCLA for graduate school.  We used both statsmodel and sklearn for comparison and confirmation.  Statsmodel enables us to see the p-value and confidence intervals for each coefficient.


> ## Question 25.  Results
>
> Write up your results.

Answer: TODO

In our final model, all of our features were statistically significant at the .05 level.  That is, GRE, GPA, and prestige of college were all important factors in predicting admission to UCLA graduate school. 

Specifically:
e^(.002)=1.001  The odds of being admitted to UCLA is slightly higher for people who have an additional point in GRE score.
e^(.96)=2.6  The odds of being admitted to UCLA is 2.6 higher for people who have an additional point in GPA
e^(-0.47) = .5.  The odds of being admitted to UCLA is half as likely if one is attending a second rank college vs a top ranked college.
e^(-0.71)=0.5  The odds of being addmitted to UCLA is half as likely if one is attending a third rank college vs a top ranked college.
e^(-0.79)=0.45 The odds of being addmitted to UCLA is less than half as likely if one is attending a third rank college vs a top ranked college.

> ## Question 26.  Visuals
>
> Provide a table or visualization of these results.

In [98]:
print result.summary()
# the following is a nice summary of the statsmodel results which shows that each of the 
# features is statistically significant at the .05 level.  

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 08 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        11:05:28   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2    -0.6801      0.317     -2.146      0.0

> ## Question 27.  Discussion
>
> Write up your discussion and future steps.

Answer: TODO

Because we do not have many features in our dataset, we are less worried about finding the correct features to include in our model.

For next steps:

1.  It would be nice to divide the GRE scores by 100 so that our interpretation of its coefficient depends on increasing GRE scores by 80 points, rather than 1 point.  It is more reasonable to see differences of 100 points in GRE scores.

2.  Split data into training and test data so that we can estimate prediction error of the model

3.  Reduce prediction error by:
   a.  regularization
   b.  feature selection


